In [9]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:


import importlib
import functions
import spectrogram_plotting_functions

importlib.reload(functions)
importlib.reload(spectrogram_plotting_functions)

<module 'spectrogram_plotting_functions' from 'c:\\Users\\sinha\\cpl_analysis_naman\\spectrogram_plotting_functions.py'>

## 02 Loading the folder that contains our data

In [11]:
import glob
import getpass
user= (getpass.getuser())
print("Hello", user)

base='C:\\Users\\{}\\Dropbox\\CPLab'.format(user)
files = glob.glob(base+'\\all_data_mat_filtered\\*.mat')
savepath = base+'\\results\\'
print(files)

Hello sinha
['C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk1_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk3_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk5_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk6_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk1_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk3_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk5_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk6_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230609_dk1_BW_nocontext_day1.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230609_dk3_BW_nocontext_day1.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230610_dk1_BW_noconte

In [12]:
keyboard_dict={'98':'b','119':'w','120':'nc','49':'1','48':'0'}

In [18]:

events_codes_all = {}

for file in files :
    if 'dk6' in file:
        print(file)    
        base_name = os.path.basename(file)
        base_name, _ = os.path.splitext(base_name)

        date, mouse_id, task=functions.exp_params(base_name)
        print(date, mouse_id, task)
        ## Open the file
        if task=='nocontextday2' or task=='nocontextos2':
            #print(i)
            task='nocontext'
            #compiled_data.at[i,'experiment']='nocontext'
        print(date, mouse_id, task)


        f=h5py.File(file, 'r')
        channels = list(f.keys())
        print(channels)

        if 'Keyboard' in channels:
            events = f['Keyboard']
        elif 'keyboard' in channels:
            events = f['keyboard']
        elif 'memory' in channels:
            events = f['memory']
        elif 'Memory' in channels:
            events = f['Memory']

        events_codes=np.array(events['codes'][0])
        events_times=np.array(events['times'][0])
        events_codes_all[base_name] = events_codes
        epochs=functions.generate_epochs_with_first_event(events_codes, events_times)

        # Experiment Start time
        first_event=events_times[0]
        for channeli in channels:
            if "AON" in channeli or  "vHp" in channeli :
                
                channel_id=channeli
                # Extracting raw data and time
                data_all=f[channeli]
                raw_data=np.array(data_all['values']).flatten()
                raw_time = np.array(data_all['times']).flatten()
                sampling_rate = int(1/data_all['interval'][0][0])
                print(raw_data.shape, raw_time.shape, sampling_rate)
                # Normalizing the data by subtracting the mean and std of data 30sec before the first event
                normalized_data,time,data_before=functions.data_normalization(raw_data,raw_time,first_event, sampling_rate)

                # Applying a notch filter
                notch_filtered_data=functions.iir_notch(normalized_data, sampling_rate,60)
                total=notch_filtered_data
                compiled_data=[]
                for i,epochi in enumerate(epochs):

                    door_timestamp = epochi[0][0]
                    trial_type = epochi[0][1]
                    dig_type = epochi[1, 1]
                    print(dig_type)
                    dig_timestamp = epochi[1, 0]
                    print(door_timestamp,trial_type,dig_timestamp,dig_type)
                    data_complete_trial=functions.extract_complete_trial_data(total,time,door_timestamp,dig_timestamp,sampling_rate)
                    faxis, Sxx = functions.frequency_domain(data_complete_trial,time)
                    compiled_data.append([i, faxis, Sxx])
                
                compiled_data_df = pd.DataFrame(compiled_data, columns=['Trial', 'Frequency Axis', 'Power Spectrum'])
                print(compiled_data_df.shape)

C:\Users\sinha\Dropbox\CPLab\all_data_mat_filtered\20230529_dk6_nocontext.mat
20230529 dk6 nocontext
20230529 dk6 nocontext
['LFP1_AON', 'LFP1_vHp', 'LFP2_AON', 'LFP2_vHp', 'LFP3_AON', 'LFP4_AON', 'Memory', 'Ref', 'Respirat', 'file']
(1301568,) (1301568,) 2000
normalizing data
notch filter applied
49.0
107.19319499999999 120.0 111.93319499999998 49.0
49.0
158.621195 120.0 160.33919999999998 49.0
49.0
195.47919499999998 120.0 197.267195 49.0
49.0
254.45319499999997 120.0 255.96319999999997 49.0
49.0
316.05319999999995 120.0 317.533195 49.0
49.0
365.403195 120.0 367.07319499999994 49.0
49.0
417.88919999999996 120.0 419.64919499999996 49.0
49.0
480.90519499999994 120.0 482.24519499999997 49.0
49.0
534.253195 120.0 535.8112 49.0
49.0
582.1751949999999 120.0 583.4351949999999 49.0
(10, 3)
(1301568,) (1301568,) 2000
normalizing data
notch filter applied
49.0
107.19319499999999 120.0 111.93319499999998 49.0
49.0
158.621195 120.0 160.33919999999998 49.0
49.0
195.47919499999998 120.0 197.267195